In [ ]:
!rm -r 'IncrementalLeraningMLDL'
!git clone "https://github.com/wAnto97/IncrementalLeraningMLDL"
from IncrementalLeraningMLDL.src.CIFAR100_dataset import MyCIFAR100
from IncrementalLeraningMLDL.src.Utils import Utils
from IncrementalLeraningMLDL.src.MyNet import MyNet
from IncrementalLeraningMLDL.src.Loss import Loss

import numpy as np
import sys
import copy
from torch.backends import cudnn
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torch.utils.data import  DataLoader

from google.colab import drive
drive.mount('/content/gdrive')

Cloning into 'IncrementalLeraningMLDL'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 942 (delta 33), reused 20 (delta 9), pack-reused 886
Receiving objects: 100% (942/942), 10.62 MiB | 6.46 MiB/s, done.
Resolving deltas: 100% (616/616), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Loading data**

In [ ]:
# Define transforms for training phase
train_transform = transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize( (0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]) # Normalizes tensor with mean and standard deviation

# Define transforms for the evaluation phase
eval_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize( (0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

training_set = MyCIFAR100('/content',train=True, n_groups=10, transform=train_transform, download=True)
test_set = MyCIFAR100('/content',train=False, n_groups=10, transform=eval_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


**Hyperparameters**

In [ ]:
DEVICE = 'cuda' # 'cuda' or 'cpu'  

BATCH_SIZE = 128      # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch strain_dataloaderize, learning rate should change by the same factor to have comparable results
LR = 0.1            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-4  # Regularization, you can keep this at the default

NUM_EPOCHS = 70             # Total number of training epochs (iterations over dataset)
STEP_SIZE = [30,45,60]      # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1                 # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

CLASSES_PER_GROUP = 10
NUM_GROUPS=10

**Utils functions**

In [ ]:
###################################################################################################
#  If joint == True loads all groups until 'group', otherwise it allocates just the group 'group' #
###################################################################################################

def create_dataloaders(training_set,test_set,group,BATCH_SIZE,type):
    if(type == 'joint'):
        train,val = training_set.get_train_val_joint(group)
        test = test_set.get_groups_joint(group)
    if(type == 'finetuning'):
        train,val = training_set.get_single_train_joint_validation(group)
        test = test_set.get_groups_joint(group)
        
    train_dataloader =  DataLoader(train,batch_size=BATCH_SIZE,drop_last=True,num_workers=4,shuffle=True)
    val_dataloader = DataLoader(val,batch_size=BATCH_SIZE,drop_last=False,num_workers=4)
    test_dataloader = DataLoader(test,batch_size=BATCH_SIZE,drop_last=False,num_workers=4)

    return train_dataloader,val_dataloader,test_dataloader

In [ ]:
def validation(val_dataloader,net,conf_matrix=False):
    net.train(False)
    running_corrects = 0
    y_pred = []
    all_labels = []
    for images, labels,_ in val_dataloader:

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = net(images)
        # Get predictions
        _, preds = torch.max(outputs.data, 1)
        # Update Corrects

        running_corrects += torch.sum(preds == labels.data).data.item()
        y_pred += list(map(lambda x : x.item(),preds))
        all_labels += list(labels)

        # Calculate Accuracy
    accuracy = running_corrects / float(len(val_dataloader.dataset))

    if(conf_matrix == True):
        all_labels = list(map(lambda label : label.item(),all_labels))
        return accuracy,confusion_matrix(y_pred,np.array(all_labels))

    return accuracy

**Main**

In [ ]:
myNet = MyNet(n_classes=CLASSES_PER_GROUP)
utils = Utils()
myLoss = Loss()
typeScheduler='multistep' # In this case it can be only set to multistep
type_model = 'finetuning'

criterion = torch.nn.CrossEntropyLoss()
train_dataloader,val_dataloader,test_dataloader = create_dataloaders(training_set,test_set,1,BATCH_SIZE,type=type_model)

for i in range(NUM_GROUPS):
    best_val_accuracy = -1
    step=i+1
    print("STARTING FineTuning WITH GROUP:\t",step)  
    n_old_classes = CLASSES_PER_GROUP*(step-1)

    if step > 1:
        myNet.update_network(best_net,CLASSES_PER_GROUP + n_old_classes,myNet.init_weights)
        train_dataloader,val_dataloader,test_dataloader = create_dataloaders(training_set,test_set,step,BATCH_SIZE,type=type_model)

    optimizer,scheduler = myNet.prepare_training(LR,MOMENTUM,WEIGHT_DECAY,STEP_SIZE,GAMMA,typeScheduler=typeScheduler)

    current_step = 0
    myNet.net = myNet.net.to(DEVICE)
    cudnn.benchmark 

    # Start iterating over the epochs
    for epoch in range(NUM_EPOCHS):
        running_correct_train = 0
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))

        # Iterate over the dataset
        myNet.net.train() # Set Network to train mode
        for images, labels,_ in train_dataloader:            
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad() # Zero-ing the gradients

            # Forward pass to the network
            outputs = myNet.net(images)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            running_correct_train += torch.sum(preds == labels.data).data.item()

            loss.backward()  # backward pass: computes gradients
            if step>1 and type_model == 'finetuning':
              myNet.freeze_neurons(n_old_classes)
            optimizer.step() # update weights based on accumulated gradients
            current_step += 1

        train_accuracy = running_correct_train/len(train_dataloader.dataset)
        myNet.net.train(False)
        val_accuracy = validation(val_dataloader,myNet.net)

        print("Accuracy on the training :\t",train_accuracy)
        print("Accuracy on the validation :\t",val_accuracy)

        
        if val_accuracy > best_val_accuracy:
            best_net = copy.deepcopy(myNet.net)
            best_train_accuracy = train_accuracy
            best_val_accuracy = val_accuracy

        # Step the scheduler
        scheduler.step() 

    #Test on the train,val e test set
    test_accuracy,test_matrix = validation(test_dataloader,best_net,conf_matrix=True)
    print("Accuracy on the test :\t",test_accuracy)
    
    utils.writeOnFileMetrics('finetuningMetrics.json', step,[best_train_accuracy,best_val_accuracy,test_accuracy,test_matrix.tolist()])
    utils.writeOnFileLosses('finetuningLosses.json',step,[[],[]])
    !cp  './finetuningLosses.json' './gdrive/My Drive/finetuningLosses.json'
    !cp  './finetuningMetrics.json' './gdrive/My Drive/finetuningMetrics.json'

STARTING FineTuning WITH GROUP:	 1
Starting epoch 1/70, LR = [0.1]
Accuracy on the training :	 0.13244444444444445
Accuracy on the validation :	 0.146
Starting epoch 2/70, LR = [0.1]
Accuracy on the training :	 0.20822222222222223
Accuracy on the validation :	 0.274
Starting epoch 3/70, LR = [0.1]
Accuracy on the training :	 0.26711111111111113
Accuracy on the validation :	 0.296
Starting epoch 4/70, LR = [0.1]
Accuracy on the training :	 0.3411111111111111
Accuracy on the validation :	 0.256
Starting epoch 5/70, LR = [0.1]
Accuracy on the training :	 0.4053333333333333
Accuracy on the validation :	 0.424
Starting epoch 6/70, LR = [0.1]
Accuracy on the training :	 0.4328888888888889
Accuracy on the validation :	 0.424
Starting epoch 7/70, LR = [0.1]
Accuracy on the training :	 0.4693333333333333
Accuracy on the validation :	 0.458
Starting epoch 8/70, LR = [0.1]
Accuracy on the training :	 0.5086666666666667
Accuracy on the validation :	 0.484
Starting epoch 9/70, LR = [0.1]
Accuracy o